In [2]:
import pandas as pd
import numpy as np

- Выброс - некоторое значение, являющееся нетипичным для определенной переменной

Первый источник выбросов - Человеческий фактор. Пример: кто то внес в БД возраст человека = 200 лет.

Вторая - Технический сбой. Пример: На датчик оказывается тепловое воздействие и он не передает информацию точно (Термометр на солнце).

Третья - конвертирование данных.

In [5]:
#Тест изделий
#Мы знаем, что наш прибор отдает значения в диапазоне: от -1 до 1
df = pd.DataFrame({
    'IZM_1' : [-1.63, 0.51, -1.33, -0.83, 0.27, 1.28, 0.36, 2.01, -0.43, -0.52],
    'IZM_2' : [0.22, 0.41, -2.07, -0.74, -0.68, -0.62, 0.83, 0.30, -0.72, -0.14],
    'IZM_3' : [-0.31, 0.41, 1.26, -0.41, -0.62, -0.76, 0.02, 0.42, -1.48, 0.71]
}, index = ['TEST#1', 'TEST#2', 'TEST#3', 'TEST#4', 'TEST#5', 'TEST#6', 'TEST#7', 'TEST#8', 'TEST#9', 'TEST#10'])
df

,IZM_1,IZM_2,IZM_3
TEST#1,-1.63,0.22,-0.31
TEST#2,0.51,0.41,0.41
TEST#3,-1.33,-2.07,1.26
TEST#4,-0.83,-0.74,-0.41
TEST#5,0.27,-0.68,-0.62
TEST#6,1.28,-0.62,-0.76
TEST#7,0.36,0.83,0.02
TEST#8,2.01,0.30,0.42
TEST#9,-0.43,-0.72,-1.48
TEST#10,-0.52,-0.14,0.71


In [8]:
#Задача: проверить Измерение 3 на наличие выбросов
#Создаем маску
mask = (df['IZM_3'] > 1) | (df['IZM_3'] < -1)

In [13]:
#Более короткий способ записать маску
mask2 = np.abs(df['IZM_3']) > 1
mask2

TEST#1     False
TEST#2     False
TEST#3      True
TEST#4     False
TEST#5     False
TEST#6     False
TEST#7     False
TEST#8     False
TEST#9      True
TEST#10    False
Name: IZM_3, dtype: bool

In [10]:
mask

TEST#1     False
TEST#2     False
TEST#3      True
TEST#4     False
TEST#5     False
TEST#6     False
TEST#7     False
TEST#8     False
TEST#9      True
TEST#10    False
Name: IZM_3, dtype: bool

In [12]:
#Получаем выбросы
df[mask]['IZM_3']

TEST#3    1.26
TEST#9   -1.48
Name: IZM_3, dtype: float64

- ф-я .describe() позволяет сделать тоже самое 

Оценивая min и max в IZM_3 мы явно видим, что выбросы есть

In [15]:
df.describe()

,IZM_1,IZM_2,IZM_3
count,10.000000,10.000000,10.000000
mean,-0.031000,-0.321000,-0.076000
std,1.140248,0.828807,0.802457
min,-1.630000,-2.070000,-1.480000
25%,-0.752500,-0.710000,-0.567500
50%,-0.080000,-0.380000,-0.145000
75%,0.472500,0.280000,0.417500
max,2.010000,0.830000,1.260000


- Теперь пройдемся по всему DF

In [17]:
mask = np.abs(df) > 1
mask

,IZM_1,IZM_2,IZM_3
TEST#1,True,False,False
TEST#2,False,False,False
TEST#3,True,True,True
TEST#4,False,False,False
TEST#5,False,False,False
TEST#6,True,False,False
TEST#7,False,False,False
TEST#8,True,False,False
TEST#9,False,False,True
TEST#10,False,False,False


Чтобы получить все наши значения с выбросами необходимо нашу маску сжать до одномерной.

In [21]:
mask1d = mask.any(axis = 1)
mask1d

TEST#1      True
TEST#2     False
TEST#3      True
TEST#4     False
TEST#5     False
TEST#6      True
TEST#7     False
TEST#8      True
TEST#9      True
TEST#10    False
dtype: bool

In [23]:
#получим необходимые строки
df[mask1d]

,IZM_1,IZM_2,IZM_3
TEST#1,-1.63,0.22,-0.31
TEST#3,-1.33,-2.07,1.26
TEST#6,1.28,-0.62,-0.76
TEST#8,2.01,0.30,0.42
TEST#9,-0.43,-0.72,-1.48


## Стратегии работы с выбросами

### Зануление

In [25]:
df[mask] = 0

In [26]:
df

,IZM_1,IZM_2,IZM_3
TEST#1,0.00,0.22,-0.31
TEST#2,0.51,0.41,0.41
TEST#3,0.00,0.00,0.00
TEST#4,-0.83,-0.74,-0.41
TEST#5,0.27,-0.68,-0.62
TEST#6,0.00,-0.62,-0.76
TEST#7,0.36,0.83,0.02
TEST#8,0.00,0.30,0.42
TEST#9,-0.43,-0.72,0.00
TEST#10,-0.52,-0.14,0.71


### Приведение выбросов к определенному значению

In [29]:
#пересоздадим датасэт
df = pd.DataFrame({
    'IZM_1' : [-1.63, 0.51, -1.33, -0.83, 0.27, 1.28, 0.36, 2.01, -0.43, -0.52],
    'IZM_2' : [0.22, 0.41, -2.07, -0.74, -0.68, -0.62, 0.83, 0.30, -0.72, -0.14],
    'IZM_3' : [-0.31, 0.41, 1.26, -0.41, -0.62, -0.76, 0.02, 0.42, -1.48, 0.71]
}, index = ['TEST#1', 'TEST#2', 'TEST#3', 'TEST#4', 'TEST#5', 'TEST#6', 'TEST#7', 'TEST#8', 'TEST#9', 'TEST#10'])
df

,IZM_1,IZM_2,IZM_3
TEST#1,-1.63,0.22,-0.31
TEST#2,0.51,0.41,0.41
TEST#3,-1.33,-2.07,1.26
TEST#4,-0.83,-0.74,-0.41
TEST#5,0.27,-0.68,-0.62
TEST#6,1.28,-0.62,-0.76
TEST#7,0.36,0.83,0.02
TEST#8,2.01,0.30,0.42
TEST#9,-0.43,-0.72,-1.48
TEST#10,-0.52,-0.14,0.71


На данном шаге все значения > 1 мы будем приводить к 1, < (-1) к (-1)

In [31]:
mask = np.abs(df) > 1

In [32]:
df[mask]

,IZM_1,IZM_2,IZM_3
TEST#1,-1.63,NaN,NaN
TEST#2,NaN,NaN,NaN
TEST#3,-1.33,-2.07,1.26
TEST#4,NaN,NaN,NaN
TEST#5,NaN,NaN,NaN
TEST#6,1.28,NaN,NaN
TEST#7,NaN,NaN,NaN
TEST#8,2.01,NaN,NaN
TEST#9,NaN,NaN,-1.48
TEST#10,NaN,NaN,NaN


- np.sign() проверяет значения. Если значение меньше нуля, то будет (-1), если больше 0, то будет 1.

In [33]:
np.sign(df)

,IZM_1,IZM_2,IZM_3
TEST#1,-1.0,1.0,-1.0
TEST#2,1.0,1.0,1.0
TEST#3,-1.0,-1.0,1.0
TEST#4,-1.0,-1.0,-1.0
TEST#5,1.0,-1.0,-1.0
TEST#6,1.0,-1.0,-1.0
TEST#7,1.0,1.0,1.0
TEST#8,1.0,1.0,1.0
TEST#9,-1.0,-1.0,-1.0
TEST#10,-1.0,-1.0,1.0


In [34]:
#С помощью нашей маски передадиим -1 и 1 в наш датафрэйм и перепишем значения
df[mask] = np.sign(df)

In [36]:
#Все наши выбросы пришли в норму
df

,IZM_1,IZM_2,IZM_3
TEST#1,-1.00,0.22,-0.31
TEST#2,0.51,0.41,0.41
TEST#3,-1.00,-1.00,1.00
TEST#4,-0.83,-0.74,-0.41
TEST#5,0.27,-0.68,-0.62
TEST#6,1.00,-0.62,-0.76
TEST#7,0.36,0.83,0.02
TEST#8,1.00,0.30,0.42
TEST#9,-0.43,-0.72,-1.00
TEST#10,-0.52,-0.14,0.71


In [37]:
df.describe()

,IZM_1,IZM_2,IZM_3
count,10.000000,10.000000,10.000000
mean,-0.064000,-0.214000,-0.054000
std,0.786401,0.620917,0.670492
min,-1.000000,-1.000000,-1.000000
25%,-0.752500,-0.710000,-0.567500
50%,-0.080000,-0.380000,-0.145000
75%,0.472500,0.280000,0.417500
max,1.000000,0.830000,1.000000


## Часть 2

In [39]:
df1 = pd.DataFrame(np.arange(30).reshape(10, 3))
df1

,0,1,2
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11
4,12,13,14
5,15,16,17
6,18,19,20
7,21,22,23
8,24,25,26
9,27,28,29


In [42]:
idx = np.random.permutation(10)

In [44]:
df1.iloc[idx]

,0,1,2
2,6,7,8
3,9,10,11
1,3,4,5
6,18,19,20
4,12,13,14
7,21,22,23
8,24,25,26
0,0,1,2
5,15,16,17
9,27,28,29


In [46]:
df1.take(idx)

,0,1,2
2,6,7,8
3,9,10,11
1,3,4,5
6,18,19,20
4,12,13,14
7,21,22,23
8,24,25,26
0,0,1,2
5,15,16,17
9,27,28,29


- sample() - вытягивает некоторые СЛУЧАЙНЫЕ строки из DF

In [47]:
df1.sample(n=4)

,0,1,2
9,27,28,29
7,21,22,23
6,18,19,20
5,15,16,17


Особенность в том, что sample повторно не вытягивает строки, которые уже вытянул
Для того, чтобы была возможность получить строку, которую ранее уже получали, 
необходимо в параметр replace = True

In [50]:
df1.sample(n=4, replace = True)

,0,1,2
7,21,22,23
1,3,4,5
5,15,16,17
5,15,16,17


# Практика

### Задача 1
Сегодня мы присоединились к стартапу, который разрабатывает сайт для бронирования отелей. Каждое посещение сайта записывается в базу данных с целью отслеживания динамики посещений. Нам поручили взять данные за январь 2022 года и дать короткий комментарий о ситуации с посещениями сайта в январе. Данные уже загружены в датафрейм _df:
![](https://ucarecdn.com/5f420399-d965-45a2-a12e-6108c1e6bdde/)
(на скриншоте первые 10 записей)

Вы вызываете _df.describe() и видите статистику о наборе данных:
![](https://ucarecdn.com/fd9bd3ee-cc2d-43a9-9d90-f38d912a4451/)
Первое, что вас насторожило - это количество значений:
![](https://ucarecdn.com/edda0964-13b6-41a2-bb4b-b442d27a9b22/)
Их всего 30, а количество дней в январе 2022 года: 31. Вы подозреваете, что есть пропуск и выполняете его поиск:


![](https://ucarecdn.com/55515319-4fa9-44a3-a4c0-a19b1eef32c1/)
Отлично, есть пропуск с которым надо поработать, но это не всё! Вы также увидели, что минимальное значение для колонки с посещениями -1:
![](https://ucarecdn.com/d3882452-a7f6-4884-8d0c-3e4f0f1671a6/)
Как количество посещений за день может быть отрицательным? С этим вопросом вы подошли к разработчику Мише, который быстро посмотрел код счётчика посещений и сказал, что это баг. На самом деле -1 это тоже, что и 0 посещений.

Только вот для нас это большая разница, если мы хотим правильно посчитать результаты. Чтобы правильно посчитать среднее количество посещений сайта за месяц, нам придется -1 заменить на 0.

Задания:

1) Миша сказал, что 17 января сайт не работал, поэтому NaN можно заменить на 0

2) -1 тоже замените на 0

3) Посчитайте среднее значение посещений за месяц

In [69]:
df = pd.DataFrame(
    {'site_visits': {'2022-01-01': 3.0, '2022-01-02': 8.0, '2022-01-03': 0.0, '2022-01-04': 9.0, 
                     '2022-01-05': 9.0, '2022-01-06': 2.0, '2022-01-07': 7.0, '2022-01-08': -1.0, 
                     '2022-01-09': 7.0, '2022-01-10': 5.0, '2022-01-11': 9.0, '2022-01-12': 1.0, 
                     '2022-01-13': 4.0, '2022-01-14': -1.0, '2022-01-15': 5.0, '2022-01-16': 5.0, 
                     '2022-01-17': None, '2022-01-18': 5.0, '2022-01-19': 6.0, '2022-01-20': 6.0, 
                     '2022-01-21': 8.0, '2022-01-22': 4.0, '2022-01-23': 1.0, '2022-01-24': 0.0, 
                     '2022-01-25': 1.0, '2022-01-26': 9.0, '2022-01-27': -1.0, '2022-01-28': 8.0, 
                     '2022-01-29': 9.0, '2022-01-30': 2.0, '2022-01-31': 8.0}})

In [54]:
df

,site_visits
2022-01-01,3.0
2022-01-02,8.0
2022-01-03,0.0
2022-01-04,9.0
2022-01-05,9.0
2022-01-06,2.0
2022-01-07,7.0
2022-01-08,-1.0
2022-01-09,7.0
2022-01-10,5.0


In [72]:
import pandas as pd

def solution(df):
    df[df['site_visits'] < 0] = 0
    df.fillna(0, inplace = True)
    return df['site_visits'].mean()

In [73]:
solution(df)

4.548387096774194

In [ ]:
#Еще один вариант решения
import pandas as pd

def solution(df):
    return df.site_visits.fillna(0).replace(-1, 0).mean()

### Задача3
Продолжаем анализировать посещения сайта, но уже в марте 2022 года:
![](https://ucarecdn.com/bbac961a-1cba-4a39-ba76-b0925dbff3ba/)
(на скриншоте первые 10 записей)
![](https://ucarecdn.com/1282fa12-5ddb-4653-bf73-5a4432a0a3f9/)
Опять отрицательные посещения! Миша держится за голову, ведь он был уверен, что полностью пофиксил баг! 
![](https://ucarecdn.com/8ca271fc-cb0f-4af3-8ac9-04ef52d3d2b2/)
(выражение Миши, когда вы рассказали ему про отрицательные посещения)
По какой-то причине счетчик начал накручивать в минус хотя посещений совсем не было (судя по логам). Договорились, что это 0 посещений.
Задание: занулите все отрицательные значения

In [75]:
import pandas as pd

def solution(df):
    df[df['site_visits'] < 0] = 0
    return df

In [ ]:
#Еще решения
import pandas as pd

def solution(_df):
    _df[np.sign(_df) < 0] = 0
    return _df

In [ ]:
import pandas as pd

def solution(_df):
    return _df.clip(0)

### Задача 3
Возвращаемся на завод в отдел контроля качества. За квартал было сделано 10 тестов, в каждом тесте по 3 измерения.

Посчитайте % тестов, в которых нет ни одного выброса. Выбросом считать значения, которые меньше -1 или больше 1.

Результат не округляйте и не добавляйте знак '%'

In [77]:
df = pd.DataFrame({'IZM_1': {'TEST #1': 0.02232427, 'TEST #2': 0.4057245, 'TEST #3': 1.43896697, 
                              'TEST #4': -0.08961166, 'TEST #5': 0.47942731, 'TEST #6': -0.1445852, 
                              'TEST #7': -1.60695384, 'TEST #8': -0.15509896, 'TEST #9': 0.59731408, 
                              'TEST #10': -0.38225769}, 
                    'IZM_2': {'TEST #1': 0.06924143, 'TEST #2': -0.00535554, 'TEST #3': 0.1668628, 
                              'TEST #4': -1.19400875, 'TEST #5': -0.34500531, 'TEST #6': 0.50825417, 
                              'TEST #7': 1.92554085, 'TEST #8': -0.46557126, 'TEST #9': 1.03750377, 
                              'TEST #10': -1.02947651}, 
                    'IZM_3': {'TEST #1': -0.8196085, 'TEST #2': 0.40953733, 'TEST #3': -0.34021751, 
                              'TEST #4': -0.91692505, 'TEST #5': -0.43249034, 'TEST #6': 0.07091598, 
                              'TEST #7': 1.26657643, 'TEST #8': -1.15994801, 'TEST #9': 1.31649045, 
                              'TEST #10': -0.6475169}})

In [78]:
df

,IZM_1,IZM_2,IZM_3
TEST #1,0.022324,0.069241,-0.819608
TEST #2,0.405724,-0.005356,0.409537
TEST #3,1.438967,0.166863,-0.340218
TEST #4,-0.089612,-1.194009,-0.916925
TEST #5,0.479427,-0.345005,-0.432490
TEST #6,-0.144585,0.508254,0.070916
TEST #7,-1.606954,1.925541,1.266576
TEST #8,-0.155099,-0.465571,-1.159948
TEST #9,0.597314,1.037504,1.316490
TEST #10,-0.382258,-1.029477,-0.647517


In [86]:
mask = np.abs(df) > 1

In [87]:
mask_1d = mask.any(axis = 1)
mask_1d

TEST #1     False
TEST #2     False
TEST #3      True
TEST #4      True
TEST #5     False
TEST #6     False
TEST #7      True
TEST #8      True
TEST #9      True
TEST #10     True
dtype: bool

In [103]:
d = 100 - df[mask_1d].count()/df.count()*100

In [104]:
d[1].item()

40.0

In [ ]:
#Мое итоговое колхозное решение
import pandas as pd

def solution(df):
    mask = np.abs(df) > 1
    mask_1d = mask.any(axis = 1)
    d = 100 - df[mask_1d].count()/df.count()*100
    return d[1].item()

In [105]:
#Ну и нормальное решение естественно
import pandas as pd

def solution(df):
    return float((df.abs() <= 1).all(axis=1).mean() * 100)